In [1]:
import PySimpleGUI as sg
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
## Import the datasets

%run classes_choice.ipynb

In [3]:
# Create dictionary mapping inputs to classes

class_dict = {"Data Science": DS_DATA,
             "Math": MATH_DATA,
             "Actuarial Science":ACT_SCI_DATA}

In [4]:
# Render a Matplotlib file from pandas dataframe from input files
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax


In [5]:
# Helper function to draw table on interface
def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side="top", fill="both", expand=1)
    return figure_canvas_agg

In [6]:
# Function that makes the display window
def Display():
    matplotlib.use("TkAgg")

    
    # Define the window layout
    layout = [
        [sg.Text("What majors are you interested in?")],
        [sg.Text('Current Major',size=(20, 1),justification='center')],
        [sg.Combo(['Data Science','Math','Actuarial Science'],default_value='Data Science',key='current')],
        [sg.Text('New Major',size=(20, 1),justification='center')],
        [sg.Combo(['Data Science','Math','Actuarial Science'],default_value='Math',key='new')],
        [sg.Text(size=(40,3), key='-OUTPUT-')],
        [sg.Canvas(key="-CANVAS-")],
        [sg.Button('Ok'), sg.Button('Quit')]
    ]

    # Create the form and show it without the plot
    window = sg.Window(
        "Major Explorer",
        layout,
        location=(0, 0),
        finalize=True,
        element_justification="center"
    )

    # Create an event loop
    while True:
        event, values = window.Read()
        
        maj_1 = values["current"]
        maj_2 = values["new"]
        n,class_df = best_classes(class_dict[maj_1],class_dict[maj_2])
        
            # Making the dataframe into a figure
        fig,ax = render_mpl_table(class_df, header_columns=0, col_width=5.0)
        fig.savefig("test.png")

        # End program if user closes window or
        # presses the OK button
        if event == sg.WIN_CLOSED or event == 'Quit':
            break
        
        elif event == "Ok":
            # Add the plot to the window
            draw_figure(window["-CANVAS-"].TKCanvas, fig)
        
        # Output a message to the window
        window['-OUTPUT-'].update(str(n) + " more classes needed to double major"  + '\n\n' + "Recommended Classes For You")
        
    # Finish up by removing from the screen
    window.Close()
    

In [7]:
Display()

TypeError: 'NoneType' object is not subscriptable